# Stereo Video Capture

For stereo video capturing, both Ricoh Theta S cameras should be started in [Live Streaming mode](http://theta360.guide/community-document/live-streaming.html#_theta_s_live_streaming_mode), and properly connected to each USB3.0 video device capture, which in turn are plugged into the computer. You may also keep the cameras connected to the computer with the USB cable for battery charging.

The `capture_stereo.launch` script will read the video streams and display the images in two `image_view` windows.

In [ ]:
roslaunch ricoh_camera capture_stereo.launch

The displayed images should look like these ones:

| bottom | top   |
|------|------|
| <img src="img/capture_example_bottom.png" />  | <img src="img/capture_example_top.png" /> |

## Nodes

### capture_stereo.py

Simple grabber application with OpenCV for two cameras.

#### Published topics

* `bottom/image_raw` ([sensor_msgs/Image](http://docs.ros.org/api/sensor_msgs/html/msg/Image.html)) <br>
The image topic containing the two raw fisheye images of the bottom camera.
* `top/image_raw` ([sensor_msgs/Image](http://docs.ros.org/api/sensor_msgs/html/msg/Image.html)) <br>
The image topic containing the two raw fisheye images of the top camera.

#### Parameters

* `video_input_bottom` (`int`, default: 0) <br>
The video device number of the bottom camera.
* `video_input_top` (`int`, default: 1) <br>
The video device number of the top camera.
* `resolution` (`int`, default: 1280) <br>
The horizontal video resolution, 1280 or 1920.
* `frequency` (`int`, default: 30) <br>
The framegrabber frequency, between 1 and 30 Hz.

## Examples

In [ ]:
roslaunch ricoh_camera capture_stereo.launch resolution:=1920

In [ ]:
roslaunch ricoh_camera capture_stereo.launch frequency:=1

## Implementation

The capture node uses [OpenCV functions for video capture](https://docs.opencv.org/3.3.1/d8/dfe/classcv_1_1VideoCapture.html).

Particularly, it uses the [grab()](https://docs.opencv.org/3.3.1/d8/dfe/classcv_1_1VideoCapture.html#ae38c2a053d39d6b20c9c649e08ff0146) and [retrieve()](https://docs.opencv.org/3.3.1/d8/dfe/classcv_1_1VideoCapture.html#a9ac7f4b1cdfe624663478568486e6712) functions instead of [read()](https://docs.opencv.org/3.3.1/d8/dfe/classcv_1_1VideoCapture.html#a473055e77dd7faa4d26d686226b292c1). Such functions are recommended in multi-camera environments, especially when the cameras do not have hardware synchronization. That is, you call [`VideoCapture::grab()`](https://docs.opencv.org/3.3.1/d8/dfe/classcv_1_1VideoCapture.html#ae38c2a053d39d6b20c9c649e08ff0146) for each camera and after that call the slower method [`VideoCapture::retrieve()`](https://docs.opencv.org/3.3.1/d8/dfe/classcv_1_1VideoCapture.html#a9ac7f4b1cdfe624663478568486e6712) to decode and get frame from each camera. This way the overhead on demosaicing or motion jpeg decompression etc. is eliminated and the retrieved frames from different cameras will be closer in time.